In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import time
import copy
import glob
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import datasets, transforms, models

In [3]:
# ✅ 데이터 루트: 아래처럼 training/validation 폴더가 바로 아래에 있어야 함
DATA_ROOT = "/content/gdrive/MyDrive/01.DS Part/99.Study/00. Searching Datasets/Kaggle(Car damage detection)"

TRAIN_DIR = os.path.join(DATA_ROOT, "training")
VAL_DIR   = os.path.join(DATA_ROOT, "validation")

# 하이퍼파라미터
IMG_SIZE     = 224
BATCH_SIZE   = 64
NUM_WORKERS  = 2
EPOCHS       = 15
LR           = 1e-4
WEIGHT_DECAY = 1e-4

# 저장 경로
BEST_PATH = "/content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/SUBJECT/WEEK2_CAR_DAMAGE_DETECTION/JR/01.Model/resnet18_damage_vs_whole_best.pth"
LAST_PATH = "/content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/SUBJECT/WEEK2_CAR_DAMAGE_DETECTION/JR/01.Model/resnet18_damage_vs_whole_last.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [4]:
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std =(0.229, 0.224, 0.225)),
])

val_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std =(0.229, 0.224, 0.225)),
])

train_ds = datasets.ImageFolder(TRAIN_DIR, transform=train_tf)
val_ds   = datasets.ImageFolder(VAL_DIR,   transform=val_tf)

print("Class mapping (train):", train_ds.class_to_idx)
print("Class mapping (val)  :", val_ds.class_to_idx)

train_loader = DataLoader(
    train_ds, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=True
)

val_loader = DataLoader(
    val_ds, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=True
)

# ✅ pretrained 권장
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# 마지막 FC를 2클래스로 교체
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", factor=0.5, patience=2
)


Class mapping (train): {'00-damage': 0, '01-whole': 1}
Class mapping (val)  : {'00-damage': 0, '01-whole': 1}
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 197MB/s]


In [5]:
def run_epoch(model, loader, train: bool):
    if train:
        model.train()
    else:
        model.eval()

    total_loss = 0.0
    correct = 0
    total = 0

    all_true = []
    all_pred = []

    for images, labels in loader:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        if train:
            optimizer.zero_grad()

        with torch.set_grad_enabled(train):
            logits = model(images)
            loss = criterion(logits, labels)

            if train:
                loss.backward()
                optimizer.step()

        total_loss += loss.item() * images.size(0)
        preds = torch.argmax(logits, dim=1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

        all_true.append(labels.detach().cpu().numpy())
        all_pred.append(preds.detach().cpu().numpy())

    avg_loss = total_loss / total
    acc = correct / total

    all_true = np.concatenate(all_true)
    all_pred = np.concatenate(all_pred)

    return avg_loss, acc, all_true, all_pred


def confusion_matrix_2class(y_true, y_pred):
    cm = np.zeros((2, 2), dtype=int)
    for t, p in zip(y_true, y_pred):
        cm[int(t), int(p)] += 1
    return cm


In [6]:
best_acc = -1.0
best_wts = copy.deepcopy(model.state_dict())

for epoch in range(1, EPOCHS + 1):
    t0 = time.time()

    train_loss, train_acc, _, _ = run_epoch(model, train_loader, train=True)
    val_loss,   val_acc,   y_t, y_p = run_epoch(model, val_loader, train=False)

    scheduler.step(val_acc)

    dt = time.time() - t0
    print(f"[Epoch {epoch:02d}/{EPOCHS}] "
          f"train loss={train_loss:.4f} acc={train_acc:.4f} | "
          f"val loss={val_loss:.4f} acc={val_acc:.4f} | {dt:.1f}s")

    # best 저장
    if val_acc > best_acc:
        best_acc = val_acc
        best_wts = copy.deepcopy(model.state_dict())
        torch.save(best_wts, BEST_PATH)
        print(f"  ✅ Best updated! saved to: {BEST_PATH}")

# last 저장
torch.save(model.state_dict(), LAST_PATH)
print("\nTraining finished.")
print("Best val acc:", best_acc)
print("Saved last to:", LAST_PATH)


[Epoch 01/15] train loss=0.2788 acc=0.8783 | val loss=0.1477 acc=0.9348 | 286.8s
  ✅ Best updated! saved to: /content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/SUBJECT/WEEK2_CAR_DAMAGE_DETECTION/JR/01.Model/resnet18_damage_vs_whole_best.pth
[Epoch 02/15] train loss=0.0921 acc=0.9663 | val loss=0.2217 acc=0.9217 | 21.8s
[Epoch 03/15] train loss=0.0568 acc=0.9832 | val loss=0.1397 acc=0.9435 | 21.1s
  ✅ Best updated! saved to: /content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/SUBJECT/WEEK2_CAR_DAMAGE_DETECTION/JR/01.Model/resnet18_damage_vs_whole_best.pth
[Epoch 04/15] train loss=0.0284 acc=0.9913 | val loss=0.1159 acc=0.9522 | 23.2s
  ✅ Best updated! saved to: /content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/SUBJECT/WEEK2_CAR_DAMAGE_DETECTION/JR/01.Model/resnet18_damage_vs_whole_best.pth
[Epoch 05/15] train loss=0.0199 acc=0.9929 | val loss=0.1678 acc=0.9370 | 23.2s
[Epoch 06/15] train loss=0.0219 acc=0.9957 | val loss=0.1519 acc=0.9500

In [7]:
model.load_state_dict(torch.load(BEST_PATH, map_location=device))
model.eval()

val_loss, val_acc, y_true, y_pred = run_epoch(model, val_loader, train=False)
cm = confusion_matrix_2class(y_true, y_pred)

print("\n[Best Model Evaluation on validation]")
print("val loss:", val_loss)
print("val acc :", val_acc)
print("Confusion Matrix (rows=true, cols=pred):\n", cm)

# 참고 해석 (class_to_idx가 {'00-damage':0, '01-whole':1} 라면)
# cm[0,0]=damage -> damage (TP)
# cm[0,1]=damage -> whole  (FN)
# cm[1,0]=whole  -> damage (FP)
# cm[1,1]=whole  -> whole  (TN)


[Best Model Evaluation on validation]
val loss: 0.13815215722374294
val acc : 0.9608695652173913
Confusion Matrix (rows=true, cols=pred):
 [[222   8]
 [ 10 220]]


In [8]:
SAVE_MIS = True
OUT_DIR  = "/content/misclassified_val"
FN_DIR   = os.path.join(OUT_DIR, "FN_damage_pred_whole")
FP_DIR   = os.path.join(OUT_DIR, "FP_whole_pred_damage")

if SAVE_MIS:
    os.makedirs(FN_DIR, exist_ok=True)
    os.makedirs(FP_DIR, exist_ok=True)

# val_ds는 ImageFolder라서 samples에 (path, label)이 있음
# DataLoader는 shuffle=False이므로 순서가 samples와 동일하다고 가정
val_samples = val_ds.samples  # list of (path, label)

# y_true, y_pred는 위에서 best 평가 결과로 얻은 배열 사용
# 일치 길이 확인
assert len(val_samples) == len(y_true) == len(y_pred)

# class_to_idx 확인해서 damage/whole 라벨 인덱스를 확정
class_to_idx = val_ds.class_to_idx
damage_idx = class_to_idx.get("00-damage", 0)
whole_idx  = class_to_idx.get("01-whole", 1)

import shutil

fn_count, fp_count = 0, 0
for (path, true_label), pred_label in zip(val_samples, y_pred):
    true_label = int(true_label)
    pred_label = int(pred_label)

    # FN: damage(1)인데 whole(0)으로 예측 => 여기서는 "damage가 0일 수도" 있으니 idx 기반으로 정의
    if true_label == damage_idx and pred_label == whole_idx:
        shutil.copy2(path, os.path.join(FN_DIR, os.path.basename(path)))
        fn_count += 1

    # FP: whole인데 damage로 예측
    if true_label == whole_idx and pred_label == damage_idx:
        shutil.copy2(path, os.path.join(FP_DIR, os.path.basename(path)))
        fp_count += 1

print("\nSaved misclassified images:")
print("FN (damage -> predicted whole):", fn_count, "=>", FN_DIR)
print("FP (whole  -> predicted damage):", fp_count, "=>", FP_DIR)



Saved misclassified images:
FN (damage -> predicted whole): 8 => /content/misclassified_val/FN_damage_pred_whole
FP (whole  -> predicted damage): 10 => /content/misclassified_val/FP_whole_pred_damage
